# 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    roc_auc_score, average_precision_score
)
from sklearn.preprocessing import LabelBinarizer

# 데이터 불러오기

In [ ]:
train_df = pd.read_csv('closedworld_train.csv')
test_df  = pd.read_csv('closedworld_test.csv')

print(train_df.shape)
print(test_df.shape)

(13300, 27)
(5700, 27)


# feature/label 분리

In [ ]:
X_train = train_df.drop(columns=["label"]).values
y_train = train_df["label"].values

X_test  = test_df.drop(columns=["label"]).values
y_test  = test_df["label"].values

print(X_train.shape)
print(X_test.shape)

(13300, 26)
(5700, 26)


# SVM 모델 정의

In [ ]:
svm = SVC(
    kernel='rbf',
    C=1.0,
    gamma='scale',
    probability=True,
    random_state=42
)

# SVM 모델 학습

In [ ]:
svm.fit(X_train, y_train)

SVC(probability=True, random_state=42)

# 예측

In [ ]:
# Test prediction
test_pred = svm.predict(X_test)
test_proba = svm.predict_proba(X_test)

# LabelBinarizer for multiclass ROC/PR
lb = LabelBinarizer()
lb.fit(y_train)
y_test_bin = lb.transform(y_test)

# Metrics
test_acc = accuracy_score(y_test, test_pred)
test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

# ROC-AUC (multiclass OvR)
test_roc_auc = roc_auc_score(
    y_test_bin, test_proba, multi_class="ovr", average="macro"
)

# PR-AUC (macro)
pr_auc_list = []
for i in range(len(lb.classes_)):
    pr_auc_list.append(
        average_precision_score(y_test_bin[:, i], test_proba[:, i])
    )
test_pr_auc_macro = np.mean(pr_auc_list)

print("\n===== [TEST RESULTS] =====")
print(f"Accuracy       : {test_acc:.4f}")
print(f"F1 (macro)     : {test_f1_macro:.4f}")
print(f"F1 (micro)     : {test_f1_micro:.4f}")
print(f"F1 (weighted)  : {test_f1_weighted:.4f}")
print(f"ROC-AUC (OvR)  : {test_roc_auc:.4f}")
print(f"PR-AUC (macro) : {test_pr_auc_macro:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, test_pred, digits=4))


===== [TEST RESULTS] =====
Accuracy       : 0.4649
F1 (macro)     : 0.4544
F1 (micro)     : 0.4649
F1 (weighted)  : 0.4544
ROC-AUC (OvR)  : 0.9544
PR-AUC (macro) : 0.5021

Classification Report:
              precision    recall  f1-score   support

           0     0.5217    0.2000    0.2892        60
           1     0.6197    0.7333    0.6718        60
           2     0.4521    0.5500    0.4962        60
           3     0.3600    0.4500    0.4000        60
           4     0.3390    0.3333    0.3361        60
           5     0.1967    0.2000    0.1983        60
           6     0.4565    0.7000    0.5526        60
           7     0.4615    0.4000    0.4286        60
           8     0.4583    0.3667    0.4074        60
           9     0.2647    0.3000    0.2812        60
          10     0.3077    0.2000    0.2424        60
          11     0.6364    0.4667    0.5385        60
          12     0.5152    0.5667    0.5397        60
          13     0.1667    0.0167    0.0303    